In [48]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, Flatten, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import Adam, Adamax
from keras.metrics import categorical_crossentropy

from keras.preprocessing.image import ImageDataGenerator


In [49]:
IMAGE_SHAPE = (224, 224)


DATA_DIR = 'D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD'
TRAINING_DATA_DIR = []
VALID_DATA_DIR = []

for fold in os.listdir(DATA_DIR):
    TRAINING_DATA_DIR.append(DATA_DIR + '/' + fold + '/' + 'train_aug')
    VALID_DATA_DIR.append(DATA_DIR + '/' + fold + '/' + 'val')
print(TRAINING_DATA_DIR)

['D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD/FOLD_1/train_aug', 'D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD/FOLD_2/train_aug', 'D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD/FOLD_3/train_aug', 'D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD/FOLD_4/train_aug', 'D:/OneDrive - Hanoi University of Science and Technology/BackUp/EDABK/AI/Data_AI_EDABK/TuTheNgu/EDABK_IR_23CLASS_5_FOLD/FOLD_5/train_aug']


In [50]:
train_generator = []
valid_generator = []

img_height,img_width=180,180

batch_size= 32

for i in range(len(os.listdir(DATA_DIR))):
    train_generator.append(tf.keras.preprocessing.image_dataset_from_directory(
        TRAINING_DATA_DIR[i],
        seed=123,
        label_mode='categorical',
        image_size=(img_height, img_width),
        batch_size=batch_size
    ))
    valid_generator.append(tf.keras.preprocessing.image_dataset_from_directory(
        VALID_DATA_DIR[i],
        seed=123,
        label_mode='categorical',
        image_size=(img_height, img_width),
        batch_size=batch_size
    ))

Found 19094 files belonging to 23 classes.
Found 2387 files belonging to 23 classes.
Found 19096 files belonging to 23 classes.
Found 2387 files belonging to 23 classes.
Found 19096 files belonging to 23 classes.
Found 2387 files belonging to 23 classes.
Found 19096 files belonging to 23 classes.
Found 2387 files belonging to 23 classes.
Found 19096 files belonging to 23 classes.
Found 2387 files belonging to 23 classes.


In [51]:
# Build model
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,
                                                              weights="imagenet",
                                                              input_shape = (img_height, img_width, 3),
                                                                pooling = "max"
                                                              )

base_model.trainable = True

x = base_model.output
x = BatchNormalization() (x)
x = Dense(1024, activation = "relu") (x)
x = Dropout(0.3) (x)
x = Dense(512, activation = "relu") (x)
x = Dropout(0.3) (x)
x = Dense(128, activation = "relu") (x)
x = Dropout(0.3) (x)
outputs = Dense(23, activation = "softmax") (x)

model = Model(inputs = base_model.input, outputs = outputs)
lr = 0.001

model.compile(optimizer = Adamax(learning_rate = lr), loss = "categorical_crossentropy", metrics = ["accuracy"])

In [52]:
history = []

In [53]:
model.save_weights('KFOLD_Eff_weights_init.h5')
print(f"FOLD {1}: ")
history.append(model.fit(x = train_generator[0], epochs = 10,validation_data = valid_generator[0]))
model.load_weights('KFOLD_Eff_weights_init.h5')

FOLD 1: 
Epoch 1/10
 13/597 [..............................] - ETA: 4:17 - loss: 3.4627 - accuracy: 0.0505

KeyboardInterrupt: 

In [ ]:
model.save_weights('KFOLD_Eff_weights_init.h5')
print(f"FOLD {i+1}: ")
history.append(model.fit(x = train_generator[1], epochs = 10,validation_data = valid_generator[1]))
model.load_weights('KFOLD_Eff_weights_init.h5')

In [ ]:
model.save_weights('KFOLD_Eff_weights_init.h5')
print(f"FOLD {i+1}: ")
history.append(model.fit(x = train_generator[2], epochs = 10,validation_data = valid_generator[2]))
model.load_weights('KFOLD_Eff_weights_init.h5')

In [ ]:
model.save_weights('KFOLD_Eff_weights_init.h5')
print(f"FOLD {i+1}: ")
history.append(model.fit(x = train_generator[3], epochs = 10,validation_data = valid_generator[3]))
model.load_weights('KFOLD_Eff_weights_init.h5')

In [ ]:
model.save_weights('KFOLD_Eff_weights_init.h5')
print(f"FOLD {i+1}: ")
history.append(model.fit(x = train_generator[4], epochs = 10,validation_data = valid_generator[4]))
model.load_weights('KFOLD_Eff_weights_init.h5')

FOLD 1: 
Epoch 1/10
597/597 [==============================] - 285s 462ms/step - loss: 0.9820 - accuracy: 0.6928 - val_loss: 0.1236 - val_accuracy: 0.9535
Epoch 2/10
532/597 [=========================>....] - ETA: 28s - loss: 0.1466 - accuracy: 0.9544

KeyboardInterrupt: 

In [ ]:
accuracy_list = []
loss_list = []
for i in range(len(os.listdir(DATA_DIR))):
    accuracy_list.append(history[i].history['accuracy'][9] * 100)
    loss_list.append(history[i].history['loss'][9])
    print(f'Accuracy and loss of Fold {i}:', (history[i].history['accuracy'][9]))
    print(f'Loss of Fold {i}:', history[i].history['loss'][9] )

print('* Overview:')
print(f'> Accuracy: {np.mean(accuracy_list)} (Độ lệch +- {np.std(accuracy_list)})')
print(f'> Loss: {np.mean(loss_list)}')

Accuracy and loss of Fold 0: 0.9563737511634827
Loss of Fold 0: 0.19870547950267792
Accuracy and loss of Fold 1: 0.9673753380775452
Loss of Fold 1: 0.16115155816078186
Accuracy and loss of Fold 2: 0.9673753380775452
Loss of Fold 2: 0.15549547970294952
Accuracy and loss of Fold 3: 0.9735546708106995
Loss of Fold 3: 0.1432795524597168
Accuracy and loss of Fold 4: 0.9695224165916443
Loss of Fold 4: 0.14426304399967194
* Overview:
> Accuracy: 96.68403029441833 (Độ lệch +- 0.569908239685569)
> Loss: 0.16057902276515962
